# Library Import

In [1]:
import duckdb
import leafmap

# Sample Data

In [ ]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

# Connecting to DuckDB

In [3]:
con = duckdb.connect('nyc_data.db')

In [4]:
con.install_extension('spatial')
con.load_extension('spatial')

In [5]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

# Intersection

In [6]:
con.sql("FROM nyc_neighborhoods SELECT * LIMIT 5;")

┌───────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────────┐
│ BORONAME  │         NAME         │                                       geom                                        │
│  varchar  │       varchar        │                                     geometry                                      │
├───────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ Brooklyn  │ Bensonhurst          │ MULTIPOLYGON (((582771.4257198056 4495167.427365481, 584651.2943549604 4497541.…  │
│ Manhattan │ East Village         │ MULTIPOLYGON (((585508.7534890148 4509691.267208001, 586826.3570590394 4508984.…  │
│ Manhattan │ West Village         │ MULTIPOLYGON (((583263.2776595836 4509242.626023987, 583276.8199068634 4509378.…  │
│ The Bronx │ Throggs Neck         │ MULTIPOLYGON (((597640.0090688139 4520272.719938631, 597647.7457808304 4520617.…  │
│ The Bronx │ Wakefield-Williams

In [7]:
con.sql("FROM nyc_subway_stations SELECT * LIMIT 5;")

┌──────────┬────────┬──────────────┬─────────────────┬───┬─────────┬─────────┬─────────┬──────────────────────┐
│ OBJECTID │   ID   │     NAME     │    ALT_NAME     │ … │  COLOR  │ EXPRESS │ CLOSED  │         geom         │
│  double  │ double │   varchar    │     varchar     │   │ varchar │ varchar │ varchar │       geometry       │
├──────────┼────────┼──────────────┼─────────────────┼───┼─────────┼─────────┼─────────┼──────────────────────┤
│      1.0 │  376.0 │ Cortlandt St │ NULL            │ … │ YELLOW  │ NULL    │ NULL    │ POINT (583521.8544…  │
│      2.0 │    2.0 │ Rector St    │ NULL            │ … │ RED     │ NULL    │ NULL    │ POINT (583324.4866…  │
│      3.0 │    1.0 │ South Ferry  │ NULL            │ … │ RED     │ NULL    │ NULL    │ POINT (583304.1823…  │
│      4.0 │  125.0 │ 138th St     │ Grand Concourse │ … │ GREEN   │ NULL    │ NULL    │ POINT (590250.1059…  │
│      5.0 │  126.0 │ 149th St     │ Grand Concourse │ … │ GREEN   │ express │ NULL    │ POINT (590454.7

Let’s find out what neighborhood the `Broad St` station is in:

In [9]:
con.sql(
    """
        SELECT subways.name AS subway_name,
               neighborhoods.name AS neighborhood_name,
               neighborhoods.boroname AS borough
        FROM nyc_neighborhoods AS neighborhoods
        JOIN nyc_subway_stations AS subways
        ON ST_Intersects(neighborhoods.geom, subways.geom)
        WHERE subways.NAME = 'Broad St';
    """
)

┌─────────────┬────────────────────┬───────────┐
│ subway_name │ neighborhood_name  │  borough  │
│   varchar   │      varchar       │  varchar  │
├─────────────┼────────────────────┼───────────┤
│ Broad St    │ Financial District │ Manhattan │
└─────────────┴────────────────────┴───────────┘

In [10]:
con.sql(
    """
        SELECT DISTINCT COLOR FROM nyc_subway_stations;
    """
)

┌─────────────────────┐
│        COLOR        │
│       varchar       │
├─────────────────────┤
│ ORANGE-YELLOW       │
│ BROWN               │
│ MULTI               │
│ GREY-ORANGE         │
│ PURPLE              │
│ ORANGE              │
│ BROWN-ORANGE        │
│ GREEN-ORANGE        │
│ GREEN-RED           │
│ BLUE-LIME           │
│     ·               │
│     ·               │
│     ·               │
│ GREY-YELLOW         │
│ PURPLE-YELLOW       │
│ SI-BLUE             │
│ RED-GREEN           │
│ BLUE-GREY           │
│ AIR-BLUE            │
│ BROWN-YELLOW        │
│ BROWN-ORANGE-YELLOW │
│ GREY                │
│ BLUE                │
├─────────────────────┤
│ 29 rows (20 shown)  │
└─────────────────────┘

Let’s find out what neighborhood the `RED` subway stations are in:

In [11]:
con.sql(
    """
        SELECT subways.name AS subway_name,
               neighborhoods.name AS neighborhood_name,
               neighborhoods.boroname AS borough
        FROM nyc_neighborhoods AS neighborhoods
        JOIN nyc_subway_stations AS subways
        ON ST_Intersects(neighborhoods.geom, subways.geom)
        WHERE subways.color = 'RED';
    """
)

┌──────────────┬──────────────────────────┬───────────┐
│ subway_name  │    neighborhood_name     │  borough  │
│   varchar    │         varchar          │  varchar  │
├──────────────┼──────────────────────────┼───────────┤
│ 242nd St     │ Riverdale                │ The Bronx │
│ 241st St     │ Wakefield-Williamsbridge │ The Bronx │
│ 238th St     │ Kings Bridge             │ The Bronx │
│ 231st St     │ Kings Bridge             │ The Bronx │
│ 225th St     │ Inwood                   │ Manhattan │
│ 215th St     │ Inwood                   │ Manhattan │
│ 207th St     │ Inwood                   │ Manhattan │
│ Dyckman St   │ Washington Heights       │ Manhattan │
│ 191st St     │ Washington Heights       │ Manhattan │
│ 181st St     │ Washington Heights       │ Manhattan │
│    ·         │    ·                     │     ·     │
│    ·         │    ·                     │     ·     │
│    ·         │    ·                     │     ·     │
│ Canal St     │ Tribeca                  │ Manh

# Distance Within

First, let’s get the baseline racial make-up of the city.

In [12]:
con.sql(
    """
        SELECT 100.0 * Sum(popn_white) / Sum(popn_total) AS white_pct,
               100.0 * Sum(popn_black) / Sum(popn_total) AS black_pct,
               Sum(popn_total) AS popn_total
        FROM nyc_census_blocks;
    """
)

┌───────────────────┬────────────────────┬────────────┐
│     white_pct     │     black_pct      │ popn_total │
│      double       │       double       │   int128   │
├───────────────────┼────────────────────┼────────────┤
│ 44.00395007628105 │ 25.546578900241613 │    8175032 │
└───────────────────┴────────────────────┴────────────┘

In [13]:
con.sql(
    """
        SELECT DISTINCT routes FROM nyc_subway_stations;
    """
)

┌────────────┐
│   ROUTES   │
│  varchar   │
├────────────┤
│ 6          │
│ G          │
│ F,G        │
│ M          │
│ F,Q        │
│ C,E        │
│ D,M        │
│ Q          │
│ B,D,E      │
│ E,F        │
│ ·          │
│ ·          │
│ ·          │
│ D          │
│ M,R        │
│ D,M,N,R    │
│ B,Q        │
│ J          │
│ B,Q,S      │
│ B,C        │
│ N,Q,R,W    │
│ E,V        │
│ N,W,7      │
├────────────┤
│  73 rows   │
│ (20 shown) │
└────────────┘

So to find the A-train, we will want any row in `routes` that has an ‘A’ in it. We can do this a number of ways, but here we will use the fact that strpos(routes, ’A’) will return a non-zero number only if ‘A’ is in the `routes` field.

In [14]:
con.sql(
    """
        SELECT DISTINCT routes
        FROM nyc_subway_stations AS subways
        WHERE strpos(subways.routes,'A') > 0;
    """
)

┌─────────┐
│ ROUTES  │
│ varchar │
├─────────┤
│ A       │
│ A,C,F   │
│ A,C,E   │
│ A,S     │
│ A,C,G   │
│ A,C,E,L │
│ A,C     │
│ A,B,C,D │
│ A,B,C   │
└─────────┘

Let’s summarize the racial make-up of within 200 meters of the A-train line

In [15]:
con.sql(
    """
        SELECT 100.0 * Sum(popn_white) / Sum(popn_total) AS white_pct,
               100.0 * Sum(popn_black) / Sum(popn_total) AS black_pct,
               Sum(popn_total) AS popn_total
        FROM nyc_census_blocks AS census
        JOIN nyc_subway_stations AS subways
        ON ST_DWithin(census.geom, subways.geom, 200)
        WHERE strpos(subways.routes,'A') > 0;
    """
)

┌───────────────────┬───────────────────┬────────────┐
│     white_pct     │     black_pct     │ popn_total │
│      double       │      double       │   int128   │
├───────────────────┼───────────────────┼────────────┤
│ 45.59012559002023 │ 22.09362356709373 │     189824 │
└───────────────────┴───────────────────┴────────────┘

# Advanced Join

In [16]:
con.sql(
    """
        CREATE OR REPLACE TABLE subway_lines ( route char(1) );
        INSERT INTO subway_lines (route) VALUES
            ('A'),('B'),('C'),('D'),('E'),('F'),('G'),
            ('J'),('L'),('M'),('N'),('Q'),('R'),('S'),
            ('Z'),('1'),('2'),('3'),('4'),('5'),('6'),
            ('7');
    """
)

In [18]:
con.sql(
    """
        SELECT * FROM subway_lines;
    """
)

┌─────────┐
│  route  │
│ varchar │
├─────────┤
│ A       │
│ B       │
│ C       │
│ D       │
│ E       │
│ F       │
│ G       │
│ J       │
│ L       │
│ M       │
│ N       │
│ Q       │
│ R       │
│ S       │
│ Z       │
│ 1       │
│ 2       │
│ 3       │
│ 4       │
│ 5       │
│ 6       │
│ 7       │
├─────────┤
│ 22 rows │
└─────────┘

In [19]:
con.sql(
    """
        SELECT lines.route,
               100.0 * Sum(popn_white) / Sum(popn_total) AS white_pct,
               100.0 * Sum(popn_black) / Sum(popn_total) AS black_pct,
               Sum(popn_total) AS popn_total
        FROM nyc_census_blocks AS census
        JOIN nyc_subway_stations AS subways
        ON ST_DWithin(census.geom, subways.geom, 200)
        JOIN subway_lines AS lines
        ON strpos(subways.routes, lines.route) > 0
        GROUP BY lines.route
        ORDER BY black_pct DESC;
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┬────────────────────┬────────────────────┬────────────┐
│  route  │     white_pct      │     black_pct      │ popn_total │
│ varchar │       double       │       double       │   int128   │
├─────────┼────────────────────┼────────────────────┼────────────┤
│ S       │ 39.839644455121466 │ 46.503108014774334 │      33301 │
│ 3       │  42.72731756087282 │  42.06198693548893 │     223047 │
│ 5       │  33.79377760724286 │  41.38562664729877 │     218919 │
│ 2       │  39.26304853922876 │  38.39114588512005 │     291661 │
│ C       │  46.87871806640494 │ 30.598767440098747 │     224411 │
│ 4       │  37.55300060572121 │ 27.428313466439615 │     174998 │
│ B       │  39.95588172248356 │ 26.852519457641385 │     256583 │
│ A       │  45.59012559002023 │  22.09362356709373 │     189824 │
│ J       │  37.62955269040576 │ 21.637651380013697 │     132861 │
│ Q       │  56.88447982881239 │  20.63141166844987 │     127112 │
│ Z       │  38.35718630567766 │  20.15700496952864 │      871

# Projection

In [20]:
url = 'https://open.gishub.org/data/duckdb/cities.parquet'
con.sql(f"SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) AS geometry FROM '{url}'")

┌─────────┬────────┬───────────┬───────────┬──────────────────┬────────────┬─────────────────────────────┐
│ country │   id   │ latitude  │ longitude │       name       │ population │          geometry           │
│ varchar │ double │  double   │  double   │     varchar      │   double   │          geometry           │
├─────────┼────────┼───────────┼───────────┼──────────────────┼────────────┼─────────────────────────────┤
│ UGA     │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │ POINT (32.5333 0.5833)      │
│ UGA     │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │ POINT (30.275 0.671)        │
│ ITA     │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │ POINT (15.799 40.642)       │
│ ITA     │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │ POINT (14.656 41.563)       │
│ ITA     │    5.0 │    45.737 │     7.315 │ Aosta            │    34062.0 │ POINT (7.315 45.737)        │
│ ALD     │    6.0 │    60.097 │    1

In [21]:
con.sql(
    f"""
        SELECT * EXCLUDE geometry, ST_Transform(ST_GeomFromWKB(geometry), 'EPSG:4326', 'EPSG:5070', true) AS geometry FROM '{url}'
    """
)

┌─────────┬────────┬───────────┬───────────┬──────────────────┬────────────┬───────────────────────────────────────────┐
│ country │   id   │ latitude  │ longitude │       name       │ population │                 geometry                  │
│ varchar │ double │  double   │  double   │     varchar      │   double   │                 geometry                  │
├─────────┼────────┼───────────┼───────────┼──────────────────┼────────────┼───────────────────────────────────────────┤
│ UGA     │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │ POINT (11942089.442723729 7279926.80195…  │
│ UGA     │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │ POINT (11867630.052040448 6998929.07050…  │
│ ITA     │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │ POINT (7358230.68251168 6866592.8408855…  │
│ ITA     │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │ POINT (7226148.687146555 6819079.567060…  │
│ ITA     │    5.0 │    45.737 │

# References

- [duckdb](https://duckdb.org/docs/archive/0.9.2/extensions/spatial#spatial-relationships)
- [Introduction to PostGIS - Spatial Joins](https://postgis.net/workshops/postgis-intro/joins.html)